In [28]:
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import patches
from matplotlib import pyplot as plt
from matplotlib import gridspec as gsp
from matplotlib.widgets import Slider, Button, RadioButtons
from ipywidgets import *

import numpy as np
import csv
import os
import glob
from functools import reduce
import SimpleITK as sitk
from scipy.spatial.distance import cdist

from skimage import measure
from skimage import filters

import time

In [2]:
def get_data(locroot, mtype):
    truthlist = glob.glob(locroot+mtype+'/005/*/predictions/seg-*-tumor.nii')
    truthlist.sort()
    predlist  = glob.glob(locroot+mtype+'/005/*/predictions/pred-*-tumor-masked-int.nii')
    predlist.sort()
    assert len(truthlist) == len(predlist)
    print('Found', len(truthlist), 'segmentations and predictions...')
    return truthlist, predlist


In [3]:
# archs = ['unet', 'resnet', 'densenet']
# pocket = ['', '-pocket']
# c2Dt = ['', '-c2Dt']
archs = ['densenet']
pocket = ['-pocket']
c2Dt = ['']
modeltypes = []
for a in archs:
    for c in c2Dt:
        for p in pocket:
            modeltypes += [a+p+c]

locroot = '/rsrch1/ip/jacctor/smallnet/tumorhcc-ensemble/densenet-ensemble-512/'
# locroot = '/rsrch1/ip/jacctor/smallnet/tumorhcc-ensemble/e2e-512/'

In [4]:
def ShowImage(img):
    data = sitk.GetArrayFromImage(img)
    plt.imshow(data)
    plt.show()

In [5]:
def SliceViewer(data, data2=None):
    img = sitk.GetArrayFromImage(data)
    cmax = np.max(img)
    
    if data2 is not None:
        seg = sitk.GetArrayFromImage(data2)
        cmax = max([cmax, np.max(seg)])

    def f(Slice):
        if data2 is not None:
            fig = plt.figure(figsize=(8,4))
            ax  = fig.add_subplot(121)
        else:
            fig = plt.figure(figsize=(4,4))
            ax  = fig.add_subplot(111)
        img0 = img[...,Slice]
        ax.imshow(img0, cmap="hsv", vmin=0, vmax=cmax)
        if data2 is not None:
            ax = fig.add_subplot(122)
            seg0 = seg[...,Slice]
            ax.imshow(seg0, cmap="hsv", vmin=0, vmax=cmax)
    interact(f, Slice=widgets.IntSlider(min=0, max=img.shape[-1],step=1,value=0))

In [6]:
def closest_node(node, nodes):
    min_idx = cdist([node], nodes).argmin()
    return int(min_idx), nodes[min_idx]

In [39]:
###
### true, pred are SITK::Image
### returns hdist_avg, hdist_max, dice, vol_true
###
def GetSITKStats(true, pred):
    
    # Calculate Hausdorff distances
    try:
        hausdorffFilter = sitk.HausdorffDistanceImageFilter()
#         hausdorffFilter.Execute(true, pred)
#         hdist_avg = hausdorffFilter.GetAverageHausdorffDistance()
#         hdist_max = hausdorffFilter.GetHausdorffDistance()
        hdist_avg = None
        hdist_max = None
    except:
        hdist_avg = None
        hdist_max = None
        
    
    # Calculate Dice scores
    try:
        diceFilter = sitk.LabelOverlapMeasuresImageFilter()
        diceFilter.Execute(true, pred)
        dice = diceFilter.GetDiceCoefficient()
    except:
        dice = None
    
    # Calculate true volume statistics
    try:
        stats = sitk.LabelShapeStatisticsImageFilter()
        stats.Execute(true)
        vol_true = stats.GetNumberOfPixels(1)
    except:
        vol_true = None
    
    results = (hdist_avg, hdist_max, dice, vol_true)
    return results


def GetNpyStats(true, pred):
    if np.sum(pred) == 0:
        dice = 0
    else:
        dice = (2 * np.sum( true * pred ) + 1.0) / ( np.sum(true) + np.sum(pred) + 1.0)
    vol_true = np.sum(true)
    
    results = (None, None, dice, vol_true)
    return results


In [8]:
def connected_components(img):
    labels, n_labels = measure.label(img, return_num=True)
    return labels, n_labels

def get_connected_component_properties(img):
    props = measure.regionprops(img)
    return props

In [34]:
def StatsNPY(truth, pred, iii):
    truth = sitk.ReadImage(truth, sitk.sitkUInt8, imageIO="NiftiImageIO")
    pred  = sitk.ReadImage(pred,  sitk.sitkUInt8, imageIO="NiftiImageIO")
    
    truth = sitk.GetArrayFromImage(truth)
    pred  = sitk.GetArrayFromImage(pred)
    
    global_scores = GetNpyStats(truth, pred)
    individual_scores = {}

    conn_comp_true, num_true_components = connected_components(truth)
    conn_comp_pred, num_pred_components = connected_components(pred)
    
    true_centroids      = np.zeros((num_true_components, 3))
    pred_centroids      = np.zeros((num_pred_components, 3))
    print('\t Found', num_true_components, 'regions in true segmentation.')
    print('\t Found', num_pred_components, 'regions in predicted segmentation.')

    individual_scores['true_vol']     = []
    individual_scores['closest-dice'] = []
    individual_scores['max-dice']     = []
    individual_scores['hdist_avg']    = []
    individual_scores['hdist_max']    = []

    
    if num_true_components == 0:
        return global_scores, individual_scores, [0.0, 0.0]
    if num_pred_components == 0:
        return global_scores, individual_scores, [0.0, 0.0]
    
        
    conn_comp_true_props = get_connected_component_properties(conn_comp_true)
    conn_comp_pred_props = get_connected_component_properties(conn_comp_pred)
    
    for idx in range(num_true_components):
        true_centroids[idx,:] = np.array( conn_comp_true_props[idx]['centroid'])

    for idx in range(num_pred_components):
        pred_centroids[idx,:] = np.array( conn_comp_pred_props[idx]['centroid'])
        
    
    dsc_matrix  = np.zeros((num_true_components, num_pred_components))
    closest_idx = np.zeros((num_true_components,), dtype=np.int16)
    maxdice_idx = np.zeros((num_true_components,), dtype=np.int16)
    for idx in range(num_true_components):
        this_truth = (conn_comp_true == (idx+1)).astype(np.uint8)
        
        pred_idx, _ = closest_node(true_centroids[idx,:], pred_centroids)
        closest_idx[idx] = pred_idx

        this_pred = (conn_comp_pred == pred_idx).astype(np.uint8)
        this_tumor_stats = GetNpyStats(this_truth, this_pred)

        individual_scores['hdist_avg']    += [this_tumor_stats[0]]
        individual_scores['hdist_max']    += [this_tumor_stats[1]]
        individual_scores['closest-dice'] += [this_tumor_stats[2]]
        individual_scores['true_vol']     += [this_tumor_stats[3]]
        
        for pred_idx in range(num_pred_components):
            this_pred = (conn_comp_pred == (pred_idx+1)).astype(np.uint8)
            this_tumor_stats = GetNpyStats(this_truth, this_pred)
            dsc_matrix[idx, pred_idx] = this_tumor_stats[2]
    
    maxdice_idx = np.argmax(dsc_matrix, axis=-1)
    maxdice     = np.max(dsc_matrix, axis=-1)
    closedice   = np.array( [dsc_matrix[idx, closest_idx[idx]] for idx in range(dsc_matrix.shape[0])])

    print('\t Global dice: {:.4f}'.format(global_scores[2]))
    for idx in range(num_true_components):
        print('\t', idx, '\tDice (closest): {:.4f}'.format(closedice[idx]), '\tDice (max): {:.4f}'.format(maxdice[idx]))
    
    gs = gsp.GridSpec(1,3, width_ratios=[num_pred_components,1,1])
    fig = plt.figure()
    ax1 = plt.subplot(gs[0])
    ax1.imshow(dsc_matrix, cmap='gray', aspect='equal')
    ax1.title.set_text('matrix')
    for idx in range(num_true_components):
        pred_idx = closest_idx[idx]
        max_idx  = maxdice_idx[idx]
        rect = patches.Rectangle((max_idx-0.5,  idx-0.5), 1, 1, edgecolor='g', facecolor="none", linewidth=3)
        ax1.add_patch(rect)
        rect = patches.Rectangle((pred_idx-0.5, idx-0.5), 1, 1, edgecolor='r', facecolor="none", linewidth=3)
        ax1.add_patch(rect)
    
    ax2 = plt.subplot(gs[1])
    ax2.imshow(maxdice[:, np.newaxis],   cmap='gray')
    ax2.title.set_text('max')
    ax2.set_aspect('equal')
    ax2.xaxis.set_visible(False)
    
    ax3 = plt.subplot(gs[2])
    ax3.imshow(closedice[:, np.newaxis], cmap='gray')
    ax3.title.set_text('closest')
    ax3.set_aspect('equal')
    ax3.xaxis.set_visible(False)
    
    plt.savefig('dice-matrices-npy/dice-matrix-'+str(iii)+'.png')
    plt.close()
    plt.clf()
    
    for idx in range(num_true_components):
        individual_scores['max-dice'] += [maxdice[idx]]
    
    # filter out the false positives, and recompute global scores
    closest_seg = np.zeros_like(truth, dtype=np.uint8)
    for idx in range(num_true_components):
        closest_seg += (conn_comp_pred == (int(closest_idx[idx])+1))
    closest_seg = (closest_seg > 0).astype(np.uint8)
    filtered_closest_stats = GetNpyStats(truth, closest_seg)
    print('\t Filtered dice (closest): {:.4f}'.format(filtered_closest_stats[2]))

        
    maxreg_seg  = np.zeros_like(truth, dtype=np.uint8)
    for idx in range(num_true_components):
        maxreg_seg += (conn_comp_pred == (int(maxdice_idx[idx])+1))
    maxreg_seg = (maxreg_seg > 0).astype(np.uint8)
    filtered_maxreg_stats  = GetNpyStats(truth, maxreg_seg)
    print('\t Filtered dice     (max): {:.4f}'.format(filtered_maxreg_stats[2]))
    
    filtered_scores = [None]*2
    filtered_scores[0] = filtered_closest_stats[2]
    filtered_scores[1] = filtered_maxreg_stats[2]
    
    return global_scores, individual_scores, filtered_scores


In [42]:
mtype = modeltypes[0]
print('Model type', mtype)
truthlist, predlist = get_data(locroot, mtype)

# truthlist = truthlist[31:33]
# predlist  = predlist[31:33]

Model type densenet-pocket
Found 131 segmentations and predictions...


In [43]:
all_start_time = time.time()
print('Calculating Hausdorff distances...')
hdist_avg = [None]*len(truthlist)
hdist_max = [None]*len(truthlist)
label_sizes = [None]*len(truthlist)
total_sizes = [None]*len(truthlist)
dice_scores = [None]*len(truthlist)
dice_max_scores = [None]*len(truthlist)
dice_closest_scores = [None]*len(truthlist)
individual_stats = [None]*len(truthlist)
for idx, (tr, pr) in enumerate(zip(truthlist, predlist)):
    print('Image', idx)

    start_time = time.time()
    global_scores, individual_scores, filtered_scores = StatsNPY(tr, pr, idx)
    end_time = time.time()
    elapsed = end_time - start_time

    hdist_avg[idx] = global_scores[0]
    hdist_max[idx] = global_scores[1]
    dice_scores[idx] = global_scores[2]
    total_sizes[idx] = global_scores[3]
    label_sizes[idx] = individual_scores['true_vol']
    individual_stats[idx] = individual_scores
    dice_closest_scores[idx] = filtered_scores[0]
    dice_max_scores[idx] = filtered_scores[1]

    print('\t Time elapsed:\t', elapsed)

all_end_time = time.time()
elapsed = all_end_time - all_start_time
print('\nTotal time elapsed:\t', elapsed)

Calculating Hausdorff distances...
Image 0
	 Found 11 regions in true segmentation.
	 Found 8 regions in predicted segmentation.
	 Global dice: 0.3992
	 0 	Dice (closest): 0.0010 	Dice (max): 0.0030
	 1 	Dice (closest): 0.6803 	Dice (max): 0.6803
	 2 	Dice (closest): 0.0076 	Dice (max): 0.0100
	 3 	Dice (closest): 0.7662 	Dice (max): 0.7662
	 4 	Dice (closest): 0.0042 	Dice (max): 0.0208
	 5 	Dice (closest): 0.5684 	Dice (max): 0.5684
	 6 	Dice (closest): 0.0036 	Dice (max): 0.0062
	 7 	Dice (closest): 0.0018 	Dice (max): 0.0028
	 8 	Dice (closest): 0.0058 	Dice (max): 0.0185
	 9 	Dice (closest): 0.0031 	Dice (max): 0.0072
	 10 	Dice (closest): 0.6157 	Dice (max): 0.6157
	 Filtered dice (closest): 0.4200
	 Filtered dice     (max): 0.4424
	 Time elapsed:	 32.542166233062744
Image 1
	 Found 12 regions in true segmentation.
	 Found 14 regions in predicted segmentation.
	 Global dice: 0.5824
	 0 	Dice (closest): 0.4099 	Dice (max): 0.4099
	 1 	Dice (closest): 0.7818 	Dice (max): 0.7818
	 2

	 Global dice: 0.3023
	 0 	Dice (closest): 0.5414 	Dice (max): 0.5414
	 Filtered dice (closest): 0.5414
	 Filtered dice     (max): 0.5414
	 Time elapsed:	 38.247852087020874
Image 19
	 Found 7 regions in true segmentation.
	 Found 5 regions in predicted segmentation.
	 Global dice: 0.5838
	 0 	Dice (closest): 0.0096 	Dice (max): 0.0096
	 1 	Dice (closest): 0.0123 	Dice (max): 0.0123
	 2 	Dice (closest): 0.0132 	Dice (max): 0.0132
	 3 	Dice (closest): 0.0333 	Dice (max): 0.2760
	 4 	Dice (closest): 0.0019 	Dice (max): 0.0019
	 5 	Dice (closest): 0.6719 	Dice (max): 0.6719
	 6 	Dice (closest): 0.0069 	Dice (max): 0.0118
	 Filtered dice (closest): 0.5471
	 Filtered dice     (max): 0.5784
	 Time elapsed:	 141.35322952270508
Image 20
	 Found 1 regions in true segmentation.
	 Found 3 regions in predicted segmentation.
	 Global dice: 0.7499
	 0 	Dice (closest): 0.8058 	Dice (max): 0.8058
	 Filtered dice (closest): 0.8058
	 Filtered dice     (max): 0.8058
	 Time elapsed:	 25.932007551193237
Im

	 Global dice: 0.7784
	 0 	Dice (closest): 0.0005 	Dice (max): 0.0051
	 1 	Dice (closest): 0.7225 	Dice (max): 0.7225
	 2 	Dice (closest): 0.8510 	Dice (max): 0.8510
	 3 	Dice (closest): 0.8827 	Dice (max): 0.8827
	 4 	Dice (closest): 0.7870 	Dice (max): 0.7870
	 5 	Dice (closest): 0.0002 	Dice (max): 0.0049
	 6 	Dice (closest): 0.0002 	Dice (max): 0.0172
	 7 	Dice (closest): 0.0004 	Dice (max): 0.0019
	 Filtered dice (closest): 0.7952
	 Filtered dice     (max): 0.7952
	 Time elapsed:	 40.73010182380676
Image 38
	 Found 0 regions in true segmentation.
	 Found 1 regions in predicted segmentation.
	 Time elapsed:	 3.953519105911255
Image 39
	 Found 1 regions in true segmentation.
	 Found 11 regions in predicted segmentation.
	 Global dice: 0.0014
	 0 	Dice (closest): 0.0003 	Dice (max): 0.0012
	 Filtered dice (closest): 0.0003
	 Filtered dice     (max): 0.0012
	 Time elapsed:	 23.363040447235107
Image 40
	 Found 17 regions in true segmentation.
	 Found 17 regions in predicted segmentatio

	 Filtered dice (closest): 0.4467
	 Filtered dice     (max): 0.4579
	 Time elapsed:	 162.7857747077942
Image 52
	 Found 2 regions in true segmentation.
	 Found 1 regions in predicted segmentation.
	 Global dice: 0.2160
	 0 	Dice (closest): 0.2408 	Dice (max): 0.2408
	 1 	Dice (closest): 0.0002 	Dice (max): 0.0002
	 Filtered dice (closest): 0.2160
	 Filtered dice     (max): 0.2160
	 Time elapsed:	 14.881158828735352
Image 53
	 Found 2 regions in true segmentation.
	 Found 5 regions in predicted segmentation.
	 Global dice: 0.3605
	 0 	Dice (closest): 0.0909 	Dice (max): 0.0909
	 1 	Dice (closest): 0.4678 	Dice (max): 0.4678
	 Filtered dice (closest): 0.4058
	 Filtered dice     (max): 0.4058
	 Time elapsed:	 9.7802734375
Image 54
	 Found 1 regions in true segmentation.
	 Found 4 regions in predicted segmentation.
	 Global dice: 0.0003
	 0 	Dice (closest): 0.0004 	Dice (max): 0.0019
	 Filtered dice (closest): 0.0004
	 Filtered dice     (max): 0.0019
	 Time elapsed:	 6.168283224105835
Imag

	 Filtered dice (closest): 0.4176
	 Filtered dice     (max): 0.7371
	 Time elapsed:	 266.10673475265503
Image 77
	 Found 2 regions in true segmentation.
	 Found 3 regions in predicted segmentation.
	 Global dice: 0.1261
	 0 	Dice (closest): 0.6513 	Dice (max): 0.6513
	 1 	Dice (closest): 0.0003 	Dice (max): 0.0003
	 Filtered dice (closest): 0.1261
	 Filtered dice     (max): 0.1272
	 Time elapsed:	 7.428604602813721
Image 78
	 Found 14 regions in true segmentation.
	 Found 21 regions in predicted segmentation.
	 Global dice: 0.5661
	 0 	Dice (closest): 0.6036 	Dice (max): 0.6036
	 1 	Dice (closest): 0.0043 	Dice (max): 0.0357
	 2 	Dice (closest): 0.8408 	Dice (max): 0.8408
	 3 	Dice (closest): 0.7144 	Dice (max): 0.7144
	 4 	Dice (closest): 0.6180 	Dice (max): 0.6180
	 5 	Dice (closest): 0.8423 	Dice (max): 0.8423
	 6 	Dice (closest): 0.0027 	Dice (max): 0.0061
	 7 	Dice (closest): 0.6841 	Dice (max): 0.6841
	 8 	Dice (closest): 0.0086 	Dice (max): 0.0141
	 9 	Dice (closest): 0.0002 	Di

	 Filtered dice (closest): 0.0953
	 Filtered dice     (max): 0.2071
	 Time elapsed:	 660.557564496994
Image 96
	 Found 0 regions in true segmentation.
	 Found 3 regions in predicted segmentation.
	 Time elapsed:	 22.82933735847473
Image 97
	 Found 9 regions in true segmentation.
	 Found 0 regions in predicted segmentation.
	 Time elapsed:	 25.85282325744629
Image 98
	 Found 30 regions in true segmentation.
	 Found 59 regions in predicted segmentation.
	 Global dice: 0.5823
	 0 	Dice (closest): 0.3034 	Dice (max): 0.5431
	 1 	Dice (closest): 0.6412 	Dice (max): 0.6412
	 2 	Dice (closest): 0.2544 	Dice (max): 0.2544
	 3 	Dice (closest): 0.0033 	Dice (max): 0.0035
	 4 	Dice (closest): 0.0051 	Dice (max): 0.0056
	 5 	Dice (closest): 0.1930 	Dice (max): 0.2429
	 6 	Dice (closest): 0.0307 	Dice (max): 0.0307
	 7 	Dice (closest): 0.6948 	Dice (max): 0.6948
	 8 	Dice (closest): 0.2092 	Dice (max): 0.5349
	 9 	Dice (closest): 0.6875 	Dice (max): 0.6875
	 10 	Dice (closest): 0.6609 	Dice (max): 

	 Filtered dice (closest): 0.1147
	 Filtered dice     (max): 0.1784
	 Time elapsed:	 28171.670429468155
Image 109
	 Found 28 regions in true segmentation.
	 Found 18 regions in predicted segmentation.
	 Global dice: 0.0548
	 0 	Dice (closest): 0.0006 	Dice (max): 0.0069
	 1 	Dice (closest): 0.1903 	Dice (max): 0.1903
	 2 	Dice (closest): 0.4634 	Dice (max): 0.4634
	 3 	Dice (closest): 0.3788 	Dice (max): 0.3788
	 4 	Dice (closest): 0.0013 	Dice (max): 0.0013
	 5 	Dice (closest): 0.0039 	Dice (max): 0.0060
	 6 	Dice (closest): 0.0016 	Dice (max): 0.0039
	 7 	Dice (closest): 0.0007 	Dice (max): 0.0008
	 8 	Dice (closest): 0.3393 	Dice (max): 0.3393
	 9 	Dice (closest): 0.3870 	Dice (max): 0.3870
	 10 	Dice (closest): 0.0086 	Dice (max): 0.0238
	 11 	Dice (closest): 0.0001 	Dice (max): 0.0001
	 12 	Dice (closest): 0.0054 	Dice (max): 0.0081
	 13 	Dice (closest): 0.1653 	Dice (max): 0.1653
	 14 	Dice (closest): 0.2749 	Dice (max): 0.2749
	 15 	Dice (closest): 0.0026 	Dice (max): 0.0033
	 1

	 Filtered dice (closest): 0.1931
	 Filtered dice     (max): 0.2565
	 Time elapsed:	 21953.968962430954
Image 118
	 Found 10 regions in true segmentation.
	 Found 38 regions in predicted segmentation.
	 Global dice: 0.1883
	 0 	Dice (closest): 0.0030 	Dice (max): 0.0038
	 1 	Dice (closest): 0.0042 	Dice (max): 0.0063
	 2 	Dice (closest): 0.0026 	Dice (max): 0.0457
	 3 	Dice (closest): 0.0041 	Dice (max): 0.0065
	 4 	Dice (closest): 0.1601 	Dice (max): 0.1601
	 5 	Dice (closest): 0.0016 	Dice (max): 0.0016
	 6 	Dice (closest): 0.0174 	Dice (max): 0.0174
	 7 	Dice (closest): 0.0029 	Dice (max): 0.0029
	 8 	Dice (closest): 0.7314 	Dice (max): 0.7314
	 9 	Dice (closest): 0.0013 	Dice (max): 0.0050
	 Filtered dice (closest): 0.0061
	 Filtered dice     (max): 0.0470
	 Time elapsed:	 658.655083656311
Image 119
	 Found 0 regions in true segmentation.
	 Found 2 regions in predicted segmentation.
	 Time elapsed:	 13.702504873275757
Image 120
	 Found 6 regions in true segmentation.
	 Found 15 reg

In [46]:
plt.figure(figsize=(12,4))
plt.scatter(list(range(len(dice_scores))), dice_scores, c='black')
plt.scatter(list(range(len(dice_closest_scores))), dice_closest_scores, c='r')
plt.scatter(list(range(len(dice_max_scores))), dice_max_scores, c='g')
plt.savefig('global-tumor-dice-results-npy.png')
plt.close()
plt.clf()

plt.figure(figsize=(4,4))
plt.hist(each_tumor_volume, bins=100, range=[0,20000])
plt.savefig('tumor-volume-hist-npy.png')
plt.close()
plt.clf()

In [47]:
dicediff_closest = np.array(dice_closest_scores) - np.array(dice_scores)
dicediff_max     = np.array(dice_max_scores)     - np.array(dice_scores)

dicediff_closest = dicediff_closest[np.isfinite(dicediff_closest)]
dicediff_max     = dicediff_max[np.isfinite(dicediff_max)]

plt.figure(figsize=(8,4))
ax = plt.subplot(121)
plt.hist(dicediff_closest, bins=50)
ax.set_title('Closest Centroid')
ax = plt.subplot(122)
plt.hist(dicediff_max, bins=50)
ax.set_title('Max Dice')
plt.savefig('global-tumor-dice-difference-filtering-npy.png')
plt.close()
plt.clf()

In [45]:
    
# Individual Dice Statistics
each_tumor_dice      = []
each_tumor_dice_max  = []
each_tumor_volume    = []
for idx in range(len(truthlist)):
    if individual_stats[idx] is not None:
        each_tumor_dice      += individual_stats[idx]['closest-dice']
        each_tumor_dice_max  += individual_stats[idx]['max-dice']
        each_tumor_volume    += individual_stats[idx]['true_vol']
print(len(each_tumor_dice))
print(len(each_tumor_volume))
print('Number of zero dice scores:', len([v for v in each_tumor_dice     if v <= 0.01]))
print('Number of zero dice scores:', len([v for v in each_tumor_dice_max if v <= 0.01]))


plt.figure(figsize=(8,4))
ax = plt.subplot(121)
plt.scatter( each_tumor_volume, each_tumor_dice)
ax.set_title('Volume vs. Dice of Closest Centroid')
plt.xlim([0,7.e4])
ax = plt.subplot(122)
plt.scatter( each_tumor_volume, each_tumor_dice_max)
ax.set_title('Volume vs. Dice of Max Predicted Region')
plt.xlim(0,7.e4)
plt.savefig('individual-tumor-dice-results-npy.png')
plt.close()
plt.clf()


879
879
Number of zero dice scores: 801
Number of zero dice scores: 354
